In [1]:
import time
from collections import Counter

Mycket av koden är hämtat från https://github.com/udacity/deep-learning/tree/master/embeddings 

In [2]:
PATH = 'data/'

In [5]:
%%time
with open(f'{PATH}dec1980.txt') as f:
    text = f.read()

CPU times: user 666 ms, sys: 469 ms, total: 1.13 s
Wall time: 1.13 s


In [6]:
# Begränsa textstorleken
#text = text[:5000000]

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [7]:
def preprocess(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <COLON> ')
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words

In [8]:
%%time 
#CPU times: user 48.3 s, sys: 7.27 s, total: 55.6 s
words = preprocess(text)
print(words[:30])

['ur', 'kb', '<COLON>', 's', 'samlingar', 'digitaliserad', 'år', '2013', 'tjanst', 'statens', 'ww', 'offentliga', 'utredningar', '1981', '<COLON>', '68', 'ey', 'kommundepartementet', 'svenska', 'kyrkans', 'gudstjänst', 'kyrkliga', 'bilaga', 'handlingar', '5', 'döden', 'döendet', '<COMMA>', 'och', 'begravningsgudstjänsten']
CPU times: user 37.6 s, sys: 9.94 s, total: 47.5 s
Wall time: 48.3 s


In [9]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 64861957
Unique words: 205965


## Make chunks

In [10]:
len(words)

64861957

In [11]:
def make_chunks (words, chunk_size):
    num_chunks = len(words)//chunk_size
    return [x * chunk_size for x in range (num_chunks)]
    

## Kapitel

In [12]:
def make_freq (words):
    counter = Counter(words)
    sum_of = sum(counter.values())
    return {x:(counter[x]/sum_of) for x in counter}

In [13]:
def compare_freqs (sub_dict, total_dict):
    return {word:(sub_dict[word]/total_dict[word]) for word in sub_dict}
    

## Kapitel

In [14]:
def characteristic_words(words, chunk_size):
    freq_total = make_freq(words)
    start_at = make_chunks(words, chunk_size)
    for x in start_at:
        words_sub = words[x: x+chunk_size]
        freq_sub = make_freq(words_sub)
        comp_freqs = compare_freqs(freq_sub, freq_total)
        list1 = sorted(comp_freqs, key=comp_freqs.get, reverse=True)
        print (list1[:10], '\n')
        

In [15]:
%%time
characteristic_words(words, 100000)

['begravningsordningarna', 'ultima', 'sakramentarium', 'commendatio', 'begravningsformel', 'förk', 'lebenslauf', 'hyresgrundande', 'kapitaibidrag', 'begravningsritualen'] 

['probiemhushåiien', 'bostadsskattefrâgor', 'bostadsskattefrågor', 'lyxvillor', 'hushåiissammansättningen', 'bostadsti11ägg', 'bostadspo1itik', 'ackvisitionsmedei', 'förmediingsverksamheten', 'villaschablonens'] 

['samförvaltningen', 'rigsmaterielexporten', 'expertrådet', 'donationskapital', 'bygdeavgift', 'fondadministrationen', 'valtningsfonden', 'förvaltningsfondens', 'lunda-fastigheterna', 'förvaltningsfonden'] 

['försvarsmaterielexportens', 'exportfaktureringen', 'und-v', 'häggl', 'färsvarsindustrin', 'exportkunder', 'utlandssam', 'fmv-beställningar', 'grovkaliberdivisionen', 'flygind'] 

['övergångenfrån', 'studiearbetspass', 'specialbetänkandet', 'undervisningsdagar', 'brukarorgan', 'framtidsaspekter', 'lövgärdet', 'dieorganisationen', 'sidoalternativ', 'studiekontraktet'] 

['slutkompetenser', 'studieprogr

['modellortsstudierna', 'tegitimation', 'sjukhuskemisten', 'scenarioalternativ', 'eg-alternativet', 'ö2', '99%-il', 'åtgärdernabör', 'minskatrafiken', 'överskridandenav'] 

['g/mile', 'ron/82', 'ron/83', 'skärholmsgymnasiet', 'remissdiskussionen', 'blytåliga', 'researchoktantal', 'minimispecifikation', '-60%', 'avgasklass'] 

['möjlighet/', 'verksamhetsmedlen', 'kulturutbytesprogram', 'kulturavtal', 'kulturattachétjänst', 'gästspelen', 'personutbytet', 'sik-propositionen', 'l985zl', 'redundant'] 

['ifip', 'strålenergin', 'etendibromid', 'bestrålningsanläggningar', 'bestrålningsanläggning', 'obestrålade', 'riskkällan', 'förskämningsbakterier', 'strålresistenta', 'scherz'] 

['kolbenstvedt', 'reproduksjon', 'bilbruk', 'trañkk', 'dårligere', 'tilgang', 'støy', 'avgiftsmärken', 'mörkerpassen', 'avläsbarheten'] 

['traktamentslös', '[kommentar', 'tjänstgöringszonen', 'kryssmarkerade', 'traktamentssumma', 'abonnemangskörning', 'abonnemangstiden', 'hemortsgräns', 'dagar-regeln', 'kr/24'] 

[

['organisationsklausulen', 'kandergård', 'avrad', 'fräise', 'kompanjonen', 'kuljak', 'jordförvärvsiagen', 'jordägandet', 'skattebonden', 'skattebondens'] 

['ãldrande', 'äldrande', 'arbetskraftsutvecklingen', 'arbetsklinik', 'delpensionsreformen', 'åldersforskningen', 'reaktionstiden', 'reaktionstid', 'åldrandets', 'äldrearbetskommitténs'] 

['non-refoulement', 'brottmålsjouren', 'facktillhörighet', 'småhustäta', 'poe-område', 'arbetartäta', 's+vpk-områden', 'l-personshushåll', 'zocumd', '_m0&lt'] 

['ulk', 'avlägsnandeförfarande', 'uat-sökande', 'arbetstillståndsprövningen', 'familjeåterföreningskvot', 'utlänningsutredningen', 'utlänningskungörelsen', 'oktoberundersökningen', 'tillstyrkta', 'au-pair'] 

['förpassningsbeslutet', 'invandringsstoppet', 'frt', 'prislagstiftningens', 'prispolitikens', 'prispoiitikens', 'wwow', '”independent', 'prispoiitiken', 'relativprisändringar'] 

['försökslagarna', 'danderyd/', 'försöksort/', '810315', 'krishnarörelsen', '801208', '820701', '840410', 

['beskattningsordningen', 'övergångsorganisationen', 'musikstiftelser', 'gionmusiken', 'musikstiftelserna', 'länsmusiken', 'musikplan', 'caprice', 'landstingsforbundets', 'musikerspeltillfâllen'] 

['internprojekten', 'fou-samordnare', 'blockverksamheten', 'fou-resultatens', 'utbildningsforskningen', 'projekthandledningen', 'planeringsrådets', 'internprojekt', 'resultatanvändning', 'fou-anslagets'] 

['projektnummer', 'hälsoindikarorer', 'vdrdefterfrdgan', 'vârdefterfrâgan', 'maxområde', 'vårdtillfallen', 'lägstavärde', 'sverigesofficiella', 'riksförsäkringsverket1977', 'diagnos/receptundersökningen'] 

['observerat/förväntat', 'studieavbrytande', 'fn=2|', 'pn=25', 'fn=60', 'fn=22', 'pn=65', 'slumpval', 'lärare/undervisning', 'm=5'] 

['adolescensen', 'storstadsskolorna', 'torsbyeleverna', 'xx00', 'landsortsskolorna', 'sko/miljöer', 'karabel', 'schale-sjöstrand-tomkinsson', 'gångsätraskolan', 'hudiksvallsskolan'] 

['processproblemen', 'näringsgrenskodade', 'ekeroth', 'iöâgornastüügare

['praktikdag', 'praktikplatsplaneringen', 'sjukgymnastutbildningens', 'rehabiliteringslinje', 'skolledningarna', 'praktikplatssamordnaren', 'huvudmannaskapskommitténs', 'grundsubvention', 'grundsubventionen', 'laboratorieassistentutbildningen'] 

['bkus', 'moá', 'mpzå', '254m', 'kapitaltering', 'kapitalutg', 'prodkostn', 'penningv', '1991-01-01', 'låneomläggningen'] 

['19811985', 'högkvaliñcerad', 'mellankvaliñcerad', '19791981', 'tandregleringsbehandling', 'gustavs', 'tandvârdsutbildningar', 'tandteknikerutbildning', '19741981', '19801981'] 

['marknadslönesystemet', 'vångdalen', 'försöksanstalterna', 'inmates‘', 'market-adjusted', 'vangdalen', 'skogomeanstalten', 'vângdalen', 'tillbergagruppen', 'skogomevistelsen'] 

['medelanstaltstid', 'medel-lång', 'mcmom', 'preimplantatorisk', 'abortinriktad', 'fostervattenprov', 'kuratorsamtal', 'fosterdiagnostiska', '18-veckorsgränsen', 'fler-handikappade'] 

['bötesgäldenärer', 'böt-', 'fängelsevite', 'följdsförändring', 'vitesgäldenär', 'böt

...

['ensamrättsuppdragets', 'provisionsuppdrag', 'salaer', 'dbkg', 'objektsbeskrivningen', 'provisjon', 'provisjonen', 'fastighetsmäklarnämnden', 'ensamrättsöverenskommelsen', 'försäkringengäller'] 

['egelnoff', 'häradsrättens', 'halda', 'gödel', 'tilda', 'slöör', 'kindblom', 'lüttners', 'scharp', 'dimming'] 

['hygienutrymmet', 'fo1kgrupp', 'folkparkema', 'stöd/kommun', 'samlingslokalhållare', 'foikgrupp', 'bygdegårdamas', '1942-1/7', 'samlinglokaler', 'lokalhållama'] 

['upa-föreningar', 'förekooperativa', 'fiv', 'uthymingar', 'hus-förening', 'foajé', 'överinsats', 'hovsta', 'studierum', 'handikapptoalett'] 

['renås', 'datapolitiskt', 'förlagsandelsinnehavarna', 'föreningschefens', 'föreningschef', 'förlagsandelsinnehavare', 'reringsmyndigheten', 'datautbildningsföretag', 'slpus', 'föreningschefen'] 

['fritidsutbildningen', 'mbf', 'dataläran', 'no-lärare', 'ommunundersökningen', 'datorarbetsplatser', 'no-lärarna', 'rörlighetskostnader', 'monopson', 'metodrättvisa'] 

['lågvinstföreta

['förmånsrätts1agen', 'förmånsrättsord-', 'massafordringar', 'iönefordringar', 'löneprivilegiet', 'kredituppföljning', 'iönegarantisystemet', 'årspersoner', 'konkursstatistiken', 'brangmo'] 

['typkalkylerna', 'standardtimmar', 'omställningsfrâgor', 'rapsfrö', 'elektrolysbaserad', 'e%', 'produktionsmâl', 'helmjölk', 'sârbarhets-', 'jem'] 

['produktionsrätter', 'slaktverksamheten', 'konkurrensberedningen', 'resursalternativet', 'produktionsmålsgruppens', 'prisstödsområde', 'maskinsamverkan', 'regleringsekonomiska', 'lnvesteringsförbudet', 'världsmarknadsprislinje'] 

['utredningsintressets', 'tvångsmedelsmyndighet', 'enmånadsregeln', 'tvångsmedelsärendet', 'tvångsmedelsingripandet', 'överskottsinformationen', 'verkställighetsintresset', 'skatteutredningsintresset', 'brottsutredningsintresset', 'anonymitetsutredningens'] 

['bieffektsfallen', 'bakgrundsuppgmer', 'genoprettelsehänsynen', 'g-h', 'tillfällighetsfynd', 'driftmedelsomslutningen', 'kvittering', 'externfmansierade', 'fakultets

['tjänstemannalagarna', 'buggning', 'extremiströrelserna', 'avlyssningsutrustning', 'blockaderna', 'kontraspionagets', 'medlemsomröstning', 'säpo-chefen', 'mikrofonen', 'l984zl9'] 

['moriveringar', 'vigningspsalmen', 'spadtagen', 'vigningsgudstjänstens', 'kyrkogårdsinvigning', 'utgångsprocessionen', 'dpr', 'liturgens', 'ärke', 'korsbäraren'] 

['bostäder-service', 'oljevärme', 'avvecklingsbeslutet', 'nattvardsdistributionen', 'transitledning', 'asar', 'delutredningen', 'medhjälparfrågan', 'diakonivigningen', 'nattvardsutdelningen'] 

['avfallstippar', 'tagene', 'torsviken', 'avfallsfri', 'ciclean', 'paradiklorbensen', 'bohuspendeln', 'paktank', 'nsvarig', 'avfallssnål'] 

['delpensionstagare', 'förköpsberättigades', 'deipensionstagare', 'deitidspensionen', 'prisregein', 'köpsberättigade', 'återköpsgarantier', 'förmögenhetstiiiväxt', 'do-myndigheten', 'bisyssieregierna'] 

['ti11stånds1agen', 'borettsiag', 'boretts1ag', 'frittstående', 'borettsiaget', 'uppgiftsbörda', 'insamlingens', '

['ill-health', 'styren/m3', '08-7878950', 'arbetsmiüörisker', 'lösningsmedlens', 'arbetsmiljöriskerna', 'lösningsmedlen', 'nervskada', 'occupationally', 'samhällsriktad'] 

['ri-systemen', 'lönerapporteringen', 'redovisningsmyndigheterna', 'typarbetsplatser', 'málhanteringen', 'målhanteringen', 'löneuträkningen', 'åklagarbiträde', 'åklagarbiträdet', 'klinikemas'] 

['åklagarregion', 'kanmaren', 'fl-examen', 'specialåklagare', 'hovrättsmål', 'brottmålsrotlar', 'undervisningsklinikerna', 'åklagarkammare', 'domarrotlar', 'lönegradsplaceringen'] 

['docentkomp', 'kretslistor', 'värdeproportionell', 'orskarutbildningens', 'kombinationsdrift', 'mandatförändringar', 'skpf', 'valkretsindelade', 'kommunproportionellt', 'elvärmeabonnenter'] 

['indelningslagskommittén', 'namnskydd', 'piratlistor', 'valsedelsstöd', 'valsedelsservice', '§vall', 'dubbelrösta', 'i§ds', '1§ds', 'mellanvalsåret'] 

['systemkultur', 'obeståndsområdet', 'fluidb', 'fridolfsson', 'obeståndstendenser', 'rekonstruktionsinsa

['grupplekarna', 'öräldramedverkan', 'områdesassistenten', 'kommunhuset', 'kanan', 'grupplekar', 'l98lz25', 'guldstjärnan', 'åldersindelade', 'barnobservationer'] 

['högskoleprovsresultat', 'svarsförslag', 'svarshäftet', 'alvarez', 'coaching', 'utforsellagen', 'förstörelsebringande', 'lllness', 'faktoranalyser', 'dtk'] 

['kyrkofullmäktigvalslagen', 'lo-kvinnorna', 'kvällstimmar', '41arb', 'sacosr-anslutna', 'landgillet', 'pastoratsindelningslagen', 'prästområdet', 'arbetstidsfdrdelning', 'obekv'] 

['tveks', 'nettoskatter', 'sb6', 'v30a', 'v30c', 'tre-skiftsarbetande', 'uppgifternaför', '40summa', '5567', 'kapitalutnyttjandetiden'] 

['institutsstyrelsen', 'länsráden', 'lnstitutsstyrelse', 'kommunikationsfunktionen', 'bostadsdelegationen', 'länsstyr-', 'fiskeenheten', 'pastoralteologiska', 'kvinnoprästmotståndarna', 'pastoralinstitut'] 

['malkiel', 'mks-89/67', '1989-08-01', '1989-08-ol', 'l/lj', 'nox-utsläppet', 'ljökonsulterna', 'mätvärdesdatorn', 'vol-%', 'uppsamlaren'] 

['s0x',

In [16]:
granska_ord = Counter(words)

In [18]:
granska_ord['konsolideringskapitalets']

9